# Robin San Roman TP DQN 



**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
!pip install sk-video
import skvideo
import skvideo.io
import cv2
import json
import random

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Input

Using TensorFlow backend.


     |████████████████████████████████| 2.3MB 4.9MB/s 


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act takes a random number x in [0,1] then if $x<\varepsilon$ the agent choose a random action otherwise it select the action according to the learn policy.  
Without epsilon the choices would ve determinitsic and thus there would be no exploration. On the contrary if epsilon is huge the agent will explore a lot and will never follow the policy that it want to learn. It is impossible to learn without exploration but it is also impossible to learn decent if we never follow the policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1  #Out of the Island
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1  #position of the mouse
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y] # The reward is the value of the board cell on which is the mouse
        self.board[self.x, self.y] = 0 # The mouse has eaten what was previously on the cell.
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over # return the new state ,the reward and a boolean for the end of the game

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2) # put cheese
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)#put poison
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array is (N+4)x(N+4) in order to handle the vision of the mouse. It has 1 for the current position of the mouse, -1 for out of the map cells, and 0 everywhere else. 

The board array has 0.5 for food -1 for poison and 0 otherwise. When the mouse is on a cell it is updated to remove what was on this cell.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        state = env.reset()
        game_over = 0
        win,lose = 0,0
        while game_over == 0 :
            prev_state =state
            action_number = agent.act(state)
            state, reward, game_over =env.act(action_number)
            
            if reward > 0:
              win = win + reward
            if reward < 0:
              lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.5/4.0. Average score (6.5)
Win/lose count 10.5/14.0. Average score (1.5)
Win/lose count 4.0/4.0. Average score (1.0)
Win/lose count 5.5/7.0. Average score (0.375)
Win/lose count 4.5/15.0. Average score (-1.8)
Win/lose count 9.0/11.0. Average score (-1.8333333333333333)
Win/lose count 9.0/13.0. Average score (-2.142857142857143)
Win/lose count 8.0/21.0. Average score (-3.5)
Win/lose count 7.5/19.0. Average score (-4.388888888888889)
Win/lose count 8.5/13.0. Average score (-4.4)
Win/lose count 13.5/14.0. Average score (-4.045454545454546)
Win/lose count 15.0/22.0. Average score (-4.291666666666667)
Win/lose count 11.0/26.0. Average score (-5.115384615384615)
Win/lose count 14.0/14.0. Average score (-4.75)
Win/lose count 5.5/6.0. Average score (-4.466666666666667)
Win/lose count 10.5/14.0. Average score (-4.40625)
Win/lose count 9.0/16.0. Average score (-4.5588235294117645)
Win/lose count 4.0/8.0. Average score (-4.527777777777778)
Win/lose count 9.5/25.0. Average score 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




With the definition :
\begin{equation*}
Q^{\pi}(s,a) = E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>
\end{equation*}
We remove the first term of the sum :
\begin{equation*}
Q^{\pi}(s,a) = E_{p^{\pi}}[ r(s_0,a_0) + \gamma\sum_{1\leq t\leq T}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>
\end{equation*}
The first term does not depends on the policy $\pi$ thus :
\begin{equation*}
Q^{\pi}(s,a) = E_{(s',a')\sim p(.|s,a)} \left( r(s,a) +\gamma E_{p^{\pi}}[ \sum_{1\leq t\leq T}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>\right)
\end{equation*}
Which is exactly what we want :

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}




Since the optimal policy is deterministic we can write : 
\begin{equation*}
Q^{\pi^*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')]
\end{equation*}

Futhermore we know that the optimal policy will maximize the expected reward, thus :
\begin{equation*}
Q^{\pi^*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma .max_{a'}Q^{\pi^*}(s',a')]
\end{equation*}


By changing side the $Q^*(s,a)$ we have :
\begin{equation*}
E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma .max_{a'}Q^{\pi^*}(s',a')- Q^{\pi^*}(s,a)] = 0
\end{equation*}

Thus we want to minimise the expectation of $r(s,a)+\gamma .max_{a'}Q^{\pi^*}(s',a')- Q^{\pi^*}(s,a) $Which leads to the objective :

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) == self.max_memory : #makes sure the memory is never out of bound 
          self.memory = self.memory[1:]
        self.memory.append(m)
        

    def random_access(self):
      return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        s_b = s[None]
        prediction = self.model.predict(s_b).argmax()
        return(prediction)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        
        steps = [self.memory.random_access() for i in range(self.batch_size)]
        
        #storing init states
        input_states = np.array([s[0] for s in steps])
        
        #storing next_states, played actions, rewards, and booleans game_over 
        next_state = np.array([s[1] for s in steps])
        tmp = np.array([s[2:] for s in steps])
        played_action = tmp[:,0].astype(int)
        reward = tmp[:,1]
        game_over = tmp[:,2]
        
        #computing current assessments of q(init state,action) for any possible action from init state
        target_q = self.model.predict(input_states)
        
        #reassessing q(init state,action) for the specific action played from init state
        q_next_state = self.model.predict(next_state) #q-values in the next state for all possible actions
        target_q_played_action = reward  + (1-game_over) * self.discount * np.max(q_next_state,axis=1) 
        for i in range(self.batch_size):
            target_q[i, played_action[i]] = target_q_played_action[i]
        
        #clipping the target to avoid exploiding gradients.
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(16, activation="relu"))
        model.add(Dense(4, activation=None))
        # NN Model        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 200, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/200 | Loss 0.0077 | Win/lose count 1.0/2.0 (-1.0)
Epoch 001/200 | Loss 0.0135 | Win/lose count 3.5/3.0 (0.5)
Epoch 002/200 | Loss 0.0073 | Win/lose count 5.5/3.0 (2.5)
Epoch 003/200 | Loss 0.0049 | Win/lose count 2.0/2.0 (0.0)
Epoch 004/200 | Loss 0.0092 | Win/lose count 6.0/5.0 (1.0)
Epoch 005/200 | Loss 0.0051 | Win/lose count 2.0/8.0 (-6.0)
Epoch 006/200 | Loss 0.0055 | Win/lose count 4.0/3.0 (1.0)
Epoch 007/200 | Loss 0.0078 | Win/lose count 1.0/3.0 (-2.0)
Epoch 008/200 | Loss 0.0089 | Win/lose count 2.0/5.0 (-3.0)
Epoch 009/200 | Loss 0.0193 | Win/lose count 2.0/1.0 (1.0)
Epoch 010/200 | Loss 0.0029 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/200 | Loss 0.0032 | Win/lose count 6.0/5.0 (1.0)
Epoch 012/200 | Loss 0.0045 | Win/lose count 4.0/2.0 (2.0)
Epoch 013/200 | Loss 0.0282 | Win/lose count 4.5/3.0 (1.5)
Epoch 014/200 | Loss 0.0114 | Win/lose count 3.0/2.0 (1.0)
Epoch 015/200 | Loss 0.0023 | Win/lose count 6.5/1.0 (5.5)
Epoch 016/200 | Loss 0.0079 | Win/lose count 4.5/2.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(filters=32, kernel_size=2, activation="relu", padding="valid", input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(filters=16, kernel_size=2, activation="relu"))
        model.add(Flatten())
        model.add(Dense(4, activation=None))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,200,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/200 | Loss 0.0017 | Win/lose count 1.0/5.0 (-4.0)
Epoch 001/200 | Loss 0.0178 | Win/lose count 1.0/3.0 (-2.0)
Epoch 002/200 | Loss 0.0033 | Win/lose count 5.0/3.0 (2.0)
Epoch 003/200 | Loss 0.0013 | Win/lose count 1.0/2.0 (-1.0)
Epoch 004/200 | Loss 0.0090 | Win/lose count 6.5/3.0 (3.5)
Epoch 005/200 | Loss 0.0007 | Win/lose count 5.5/4.0 (1.5)
Epoch 006/200 | Loss 0.0080 | Win/lose count 2.0/3.0 (-1.0)
Epoch 007/200 | Loss 0.0083 | Win/lose count 4.5/6.0 (-1.5)
Epoch 008/200 | Loss 0.0053 | Win/lose count 5.5/7.0 (-1.5)
Epoch 009/200 | Loss 0.0084 | Win/lose count 2.5/2.0 (0.5)
Epoch 010/200 | Loss 0.0054 | Win/lose count 4.5/3.0 (1.5)
Epoch 011/200 | Loss 0.0046 | Win/lose count 6.0/3.0 (3.0)
Epoch 012/200 | Loss 0.0025 | Win/lose count 4.5/3.0 (1.5)
Epoch 013/200 | Loss 0.0052 | Win/lose count 9.0/7.0 (2.0)
Epoch 014/200 | Loss 0.0027 | Win/lose count 7.0/2.0 (5.0)
Epoch 015/200 | Loss 0.0028 | Win/lose count 4.0/4.0 (0.0)
Epoch 016/200 | Loss 0.0106 | Win/lose count 8.0/2

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

I do not really understand why in this part scores are much lower during test than during train.. Anyway both in train and test the CNN is always better regardless of the temperature. The higher the temperature the more food there is on the map thus the highest score for both. We can see that the mouse often gets stuck in some places.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.6)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 38.0/3.0. Average score (35.0)
Win/lose count 41.5/8.0. Average score (34.25)
Win/lose count 26.0/3.0. Average score (30.5)
Win/lose count 25.0/1.0. Average score (28.875)
Win/lose count 37.0/4.0. Average score (29.7)
Win/lose count 33.0/4.0. Average score (29.583333333333332)
Win/lose count 33.5/4.0. Average score (29.571428571428573)
Win/lose count 36.5/5.0. Average score (29.8125)
Win/lose count 17.5/6.0. Average score (27.77777777777778)
Win/lose count 36.0/4.0. Average score (28.2)
Win/lose count 41.0/6.0. Average score (28.818181818181817)
Win/lose count 25.0/3.0. Average score (28.25)
Win/lose count 27.5/4.0. Average score (27.884615384615383)
Win/lose count 13.5/3.0. Average score (26.642857142857142)
Win/lose count 32.5/3.0. Average score (26.833333333333332)
Win/lose count 19.0/1.0. Average score (26.28125)
Win/lose count 33.5/3.0. Average score (26.529411764705884)
Win/lose count 33.0/2.0. Average score (26.77777777777778)
Win/lose count 37.5/6

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,epsilon_decay,prefix='') : 
  # decay_interval is the number of epochs between updates 
  # when we update epsilon the new value is epsilon *epsilon_decay 
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        loss = 0
        win = 0
        lose = 0

        time = 0
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss += agent.reinforce(prev_state, state,  action, reward, game_over)
            time +=1
        loss /= time
        #update epsilon
        
        agent.set_epsilon(agent.epsilon * epsilon_decay)
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, round(win,2), round(lose,2), round(win-lose,2)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1  #Out of the Island
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1  #position of the mouse
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
          reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1 #already visited malus

        reward = reward + self.board[self.x, self.y]

        self.board[self.x, self.y] = 0 # The mouse has eaten what was previously on the cell.
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over # return the new state ,the reward and a boolean for the end of the game

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2) # put cheese
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)#put poison
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
'''  
## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)'''

'  \n## use those samples of code:\n#In train explore:\nstate, reward, game_over = env.act(action, train=True)\n\n## In Environment exploring:\n# You will have to change n_state to 3 because you will use one more layer!\nreward = 0\nif train:\n    reward = -self.malus_position[self.x, self.y]\nself.malus_position[self.x, self.y] = 0.1\n\nreward = reward + self.board[self.x, self.y]\n# 3 "feature" states instead of 2\nstate = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),\n                                self.board.reshape(self.grid_size, self.grid_size,1),\n                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)'

In [0]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)

agent = DQN_CNN(size, lr=.1, epsilon = 0.95, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 200, 0.05, prefix='cnn_train_explore')

Epoch 000/200 | Loss 0.0225 | Win/lose count 5.5/28.6 (-23.1)
Epoch 001/200 | Loss 0.0130 | Win/lose count 11.2/17.5 (-6.3)
Epoch 002/200 | Loss 0.0101 | Win/lose count 6.8/19.1 (-12.3)
Epoch 003/200 | Loss 0.0098 | Win/lose count 7.7/18.0 (-10.3)
Epoch 004/200 | Loss 0.0091 | Win/lose count 2.5/20.5 (-18.0)
Epoch 005/200 | Loss 0.0064 | Win/lose count 2.0/19.6 (-17.6)
Epoch 006/200 | Loss 0.0064 | Win/lose count 7.9/18.1 (-10.2)
Epoch 007/200 | Loss 0.0060 | Win/lose count 7.6/18.1 (-10.5)
Epoch 008/200 | Loss 0.0059 | Win/lose count 2.9/19.4 (-16.5)
Epoch 009/200 | Loss 0.0070 | Win/lose count 15.5/15.6 (-0.1)
Epoch 010/200 | Loss 0.0088 | Win/lose count 6.8/18.4 (-11.6)
Epoch 011/200 | Loss 0.0085 | Win/lose count 7.8/17.6 (-9.8)
Epoch 012/200 | Loss 0.0077 | Win/lose count 7.3/18.2 (-10.9)
Epoch 013/200 | Loss 0.0066 | Win/lose count 9.6/19.6 (-10.0)
Epoch 014/200 | Loss 0.0089 | Win/lose count 7.6/18.2 (-10.6)
Epoch 015/200 | Loss 0.0087 | Win/lose count 4.1/19.1 (-15.0)
Epoch 016

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 6.0/0. Average score (6.0)
Win/lose count 13.5/1.0. Average score (9.25)
Win/lose count 8.0/0. Average score (8.833333333333334)
Win/lose count 15.5/2.0. Average score (10.0)
Win/lose count 14.5/0. Average score (10.9)
Win/lose count 12.5/1.0. Average score (11.0)
Win/lose count 1.0/1.0. Average score (9.428571428571429)
Win/lose count 5.0/0. Average score (8.875)
Win/lose count 13.0/0. Average score (9.333333333333334)
Win/lose count 11.0/1.0. Average score (9.4)
Win/lose count 17.0/0. Average score (10.090909090909092)
Win/lose count 13.5/0. Average score (10.375)
Win/lose count 10.0/0. Average score (10.346153846153847)
Win/lose count 15.0/0. Average score (10.678571428571429)
Win/lose count 3.0/0. Average score (10.166666666666666)
Win/lose count 8.0/0. Average score (10.03125)
Win/lose count 6.0/0. Average score (9.794117647058824)
Win/lose count 0.5/1.0. Average score (9.222222222222221)
Win/lose count 7.0/0. Average score (9.105263157894736)
Win/lose count 6.0/0. 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***